In [ ]:
dataColumnName = 'valueStdScaled'
model_exp = 'VARMA'

In [ ]:
if 'T' not in globals():
    T = 15
if 'predict_ahead' not in globals():
    predict_ahead = 60

In [ ]:
%run 'prepareDataSet.ipynb'

In [ ]:
%run 'utils_anomaly_detection.ipynb'

In [ ]:
import matplotlib.pyplot as plt
#%matplotlib notebook
%matplotlib inline

In [ ]:
import statsmodels.api as sm
from itertools import product

In [ ]:
#from statsmodels.graphics.tsaplots import plot_acf, plot_pacf, plot_predict
#from statsmodels.tsa.statespace.varmax import VARMAX
from statsmodels.tsa.api import VAR
from sklearn.metrics import r2_score, mean_absolute_percentage_error, mean_squared_error, mean_absolute_error

In [ ]:
X_test = dataFrame_test.to_frame()

In [ ]:
X_test['TimeLatencyStdScaled'] = dataFrameLatency_test

In [ ]:
X_test.head()

In [ ]:
X_test.shape

In [ ]:
X_test_2 = X_test.copy()

In [ ]:
cols = ['valueStdScaled', 'TimeLatencyStdScaled']

In [ ]:
X_train = dataFrame_train.to_frame()
X_train['TimeLatencyStdScaled'] = dataFrameLatency_train

In [ ]:
X_train.head()

In [ ]:
# VAR requires 2 time series. We have both response time and throughput available
# We also set the maxlags with the same value we use for the neural nets, namely T. 
# This means it will use the previous T values to predicts the next one.
# This is useful as we always have the real value (the T+1th value) so we can compare it with the prediction

In [ ]:
%%time
model_VAR = VAR(X_train)

In [ ]:
lag_order_results = model_VAR.select_order(maxlags=T)
lag_order_results

In [ ]:
lag_order_results.selected_orders

In [ ]:
results_VAR = model_VAR.fit(maxlags=T)

In [ ]:
lag_order = results_VAR.k_ar
lag_order # ensuring the maxlags is unchanged after training

In [ ]:
# This function generates the forecasts values for the test dataset, one step at a time
def generate_forecasts(x_test, model_VAR, T):
    forecasts_VAR = []
    for i in range(x_test.shape[0]-T): # we can't go past T and we start predicting the T+1 value
        prior = x_test[i:i+T][['valueStdScaled','TimeLatencyStdScaled']].to_numpy()
        fcast_VAR = model_VAR.forecast(prior, 1)
        forecasts_VAR.append(fcast_VAR)
        i += 1
    
    return forecasts_VAR

In [ ]:
# This function generates the forecasts values for the test dataset
# It predicts predict_ahead steps before getting a new ground truth value and predicts the next batch
def generate_nsteps_forecasts(x_test, model_VAR, T, predict_ahead):    
    y_predict = []
    index = 0
    while (index < x_test.shape[0]-T):
    #while len(y_predict) < x_test.shape[0]-T:
        #print(f'Selecting input from index {index} to {index+T}')
        last_x = x_test[index: index+T]
        p = model_VAR.forecast(last_x, predict_ahead)
        y_predict.append(p)
        index += predict_ahead

    y_pred_conc = y_predict[0]
    i=1
    while i < len(y_predict):
        y_pred_conc = np.concatenate((y_pred_conc, y_predict[i]), axis=0)
        i+=1

    return y_pred_conc

In [ ]:
type(X_test_2)

In [ ]:
X_test_valueStdScaled2 = []
Y_test_valueStdScaled2 = []

(X_test_valueStdScaled2, Y_test_valueStdScaled2) = formatData(X_test_valueStdScaled2, Y_test_valueStdScaled2, 
                                                          X_test_2, T, 2)

In [ ]:
Y_test_valueStdScaled2.shape, X_test_valueStdScaled2.shape

In [ ]:
X_test_valueStdScaled2[0]

In [ ]:
y_pred = generate_forecasts(X_test, results_VAR, T)

In [ ]:
y_pred_n_steps = generate_nsteps_forecasts(X_test_2.to_numpy(), results_VAR, T, predict_ahead)

In [ ]:
len(y_pred), len(y_pred_n_steps)

In [ ]:
X2 = np.array(y_pred_n_steps).reshape(-1,2)
XX2 = pd.DataFrame(X2, columns=['valueStdScaled', 'TimeLatencyStdScaled'])

In [ ]:
XX2['valueStdScaled'][0], XX2['TimeLatencyStdScaled'][0]

In [ ]:
def expand_dataframe_with_nsteps(expand_df, steps):
    for i in range(steps):
        new_row = []
        new_row.insert(0,{'valueStdScaled': expand_df['valueStdScaled'][0], 
                          'TimeLatencyStdScaled': expand_df['TimeLatencyStdScaled'][0]})
        nr = pd.DataFrame(new_row)
        expand_df = pd.concat([nr, expand_df], ignore_index=True)  
    
    return expand_df

In [ ]:
shape_dif = X_test.shape[0] - XX2.shape[0] 

In [ ]:
shape_dif

In [ ]:
XX2_new = None

In [ ]:
if shape_dif > 0:
    XX2_new = expand_dataframe_with_nsteps(XX2, shape_dif)
elif shape_dif < 0:
    XX2_new = XX2[0:X_test.shape[0]]
else:
    XX2_new = XX2.copy()

In [ ]:
XX2.shape, XX2_new.shape

In [ ]:
X_test.shape

In [ ]:
XX2_new.set_index(X_test.index, inplace=True)

In [ ]:
forecasts_VAR = generate_forecasts(X_test, results_VAR, T)
X = np.array(forecasts_VAR).reshape(-1,2)

In [ ]:
XX = pd.DataFrame(X, columns=['valueStdScaled', 'TimeLatencyStdScaled'])

In [ ]:
XX.shape

In [ ]:
XX_new = None

In [ ]:
shape_dif = X_test.shape[0] - XX.shape[0] 

In [ ]:
if shape_dif > 0:
    XX_new = expand_dataframe_with_nsteps(XX, shape_dif)
elif shape_dif < 0:
    XX_new = XX2[0:X_test.shape[0]]
else:
    XX_new = XX.copy()

In [ ]:
XX_new.shape, shape_dif

In [ ]:
XX = XX_new.copy()

In [ ]:
#for i in range(T):
#    new_row = []
#    new_row.insert(0,{'valueStdScaled': XX['valueStdScaled'][0], 'TimeLatencyStdScaled': XX['TimeLatencyStdScaled'][1]})
#    nr = pd.DataFrame(new_row)
#    XX = pd.concat([nr, XX], ignore_index=True)

In [ ]:
XX.set_index(X_test.index, inplace=True)

In [ ]:
X_test.head()

In [ ]:
X_test.shape

In [ ]:
# These are the predictions one step at a time always looking at the ground truth
X_test['valueStdScaledVarForecast'] = XX['valueStdScaled']
X_test['TimeLatencyStdScaledVarForecast'] = XX['TimeLatencyStdScaled']

In [ ]:
# These are the predictions predict_ahead steps at a time
X_test['valueStdScaledVarForecast2'] = XX2_new['valueStdScaled']
X_test['TimeLatencyStdScaledVarForecast2'] = XX2_new['TimeLatencyStdScaled']

In [ ]:
X_test.head()

In [ ]:
errors_ae = calculate_absolute_prediction_errors(X_test['valueStdScaled'].to_numpy(), X_test['valueStdScaledVarForecast'].to_numpy())
anomalies_ae = calculate_3sigma_anomalies(errors_ae)
errors_se = calculate_squared_prediction_errors(X_test['valueStdScaled'].to_numpy(), X_test['valueStdScaledVarForecast'].to_numpy())
anomalies_se = calculate_3sigma_anomalies(errors_se)

anomalies_3sigma_Y_test = calculate_3sigma_anomalies(X_test['valueStdScaled'].to_numpy())
anomalies_3sigma_y_predict = calculate_3sigma_anomalies(X_test['valueStdScaledVarForecast'].to_numpy())

anomalies_Y_test, z_scores_Y_test = calculate_zscore_anomalies(X_test['valueStdScaled'].to_numpy())
anomalies_y_predict, z_scores_y_predict = calculate_zscore_anomalies(X_test['valueStdScaledVarForecast'].to_numpy())
anomalies_errors_ae, z_scores_errors_ae = calculate_zscore_anomalies(errors_ae)
anomalies_errors_se, z_scores_errors_se = calculate_zscore_anomalies(errors_se)

anomalies_Y_test_mod, z_scores_Y_test_mod = calculate_modified_zscore_anomalies(X_test['valueStdScaled'].to_numpy())
anomalies_y_predict_mod, z_scores_y_predict_mod = calculate_modified_zscore_anomalies(X_test['valueStdScaledVarForecast'].to_numpy())
anomalies_errors_ae_mod, z_scores_errors_ae_mod = calculate_modified_zscore_anomalies(errors_ae)
anomalies_errors_se_mod, z_scores_errors_se_mod = calculate_modified_zscore_anomalies(errors_se)


In [ ]:
%run 'utils_anomaly_detection.ipynb'

In [ ]:
calculate_modified_zscore_anomalies(X_test['valueStdScaled'])[0]

In [ ]:
fig = plt.figure(figsize=(20,15))
plt.title("Anomalies Y_test")
plt.plot(X_test['valueStdScaled'].to_numpy(),label="Original Data", alpha=0.6, c='gray')
plt.scatter(np.where(anomalies_Y_test==True)[0], X_test['valueStdScaled'].to_numpy()[np.where(anomalies_Y_test==1)], 
            alpha=0.8, color='green', s=250, label="Z-Score Anomalies")
plt.scatter(np.where(anomalies_3sigma_Y_test==True)[0], X_test['valueStdScaled'].to_numpy()[np.where(anomalies_3sigma_Y_test==1)], 
            alpha=0.8, color='red', s=150, label="3-Sigma Anomalies")
plt.scatter(np.where(anomalies_Y_test_mod==True)[0], X_test['valueStdScaled'].to_numpy()[np.where(anomalies_Y_test_mod==1)], 
            alpha=0.8, color='blue', s=100, label="Modified Z-Score Anomalies")    
plt.legend()
figName = f"Y_test_anomalies-T_{T}.png"
#mlflow.log_figure(fig, figName)
plt.savefig(figName, transparent=False)
#fig.clf()
#plt.close()

In [ ]:
fig = plt.figure(figsize=(20,15))
plt.title("Predict Anomalies T=" + str(T) + " with predict 1 on "+ str(model_exp))
plt.plot(X_test['valueStdScaledVarForecast'].to_numpy(),label="Predict 1-step Forecast", alpha=0.6, c='red', linewidth=3)
plt.plot(X_test['valueStdScaled'].to_numpy(),label="Original Data", alpha=0.6, c='black')
plt.scatter(np.where(anomalies_ae==True), X_test['valueStdScaledVarForecast'].to_numpy()[np.where(anomalies_ae==True)], 
            alpha=0.8, color='green', s=350, label="3-Sigma Anomalies AE")
plt.scatter(np.where(anomalies_se==True), X_test['valueStdScaledVarForecast'].to_numpy()[np.where(anomalies_se==True)], 
            alpha=0.8, color='magenta', s=300, label = "3-Sigma Anomalies SE")
plt.scatter(np.where(anomalies_errors_ae==True), X_test['valueStdScaledVarForecast'].to_numpy()[np.where(anomalies_errors_ae==True)], 
            alpha=0.8, color='blue', s=250, label = "Z-score Anomalies AE")
plt.scatter(np.where(anomalies_errors_se==True), X_test['valueStdScaledVarForecast'].to_numpy()[np.where(anomalies_errors_se==True)], 
            alpha=0.8, color='cyan', s=200, label = "Z-score Anomalies SE")
plt.scatter(np.where(anomalies_errors_ae_mod==True), X_test['valueStdScaledVarForecast'].to_numpy()[np.where(anomalies_errors_ae_mod==True)], 
            alpha=0.8, color='lightgreen', s=150, label = "Modified Z-score Anomalies AE")
plt.scatter(np.where(anomalies_errors_se_mod==True), X_test['valueStdScaledVarForecast'].to_numpy()[np.where(anomalies_errors_se_mod==True)], 
            alpha=0.8, color='orange', s=50, label = "Modified Z-score Anomalies SE")    
plt.legend()    
figName = f"Y_predict-1-step-anomalies-T_{T}.png"
#mlflow.log_figure(fig, figName)
plt.savefig(figName, transparent=False)


In [ ]:
errors_ae2 = calculate_absolute_prediction_errors(X_test['valueStdScaled'].to_numpy(), X_test['valueStdScaledVarForecast2'].to_numpy())
anomalies_ae2 = calculate_3sigma_anomalies(errors_ae2)        
errors_se2 = calculate_squared_prediction_errors(X_test['valueStdScaled'].to_numpy(), X_test['valueStdScaledVarForecast2'].to_numpy())
anomalies_se2 = calculate_3sigma_anomalies(errors_se2)
anomalies_y_pred_nsteps_mod, z_scores_y_pred_nsteps_mod = calculate_modified_zscore_anomalies(X_test['valueStdScaledVarForecast2'].to_numpy())
anomalies_errors_ae2_mod, z_scores_errors_ae2_mod = calculate_modified_zscore_anomalies(errors_ae2)
anomalies_errors_se2_mod, z_scores_errors_se2_mod = calculate_modified_zscore_anomalies(errors_se2)

anomalies_3sigma_y_pred_nsteps = calculate_3sigma_anomalies(X_test['valueStdScaledVarForecast2'].to_numpy())
anomalies_y_pred_nsteps, z_scores_y_pred_nsteps = calculate_zscore_anomalies(X_test['valueStdScaledVarForecast2'].to_numpy())
anomalies_errors_ae2, z_scores_errors_ae2 = calculate_zscore_anomalies(errors_ae2)
anomalies_errors_se2, z_scores_errors_se2 = calculate_zscore_anomalies(errors_se2)


In [ ]:
fig = plt.figure(figsize=(20,15))        
plt.title("Predict Anomalies T=" + str(T) + " with predict " + str(predict_ahead) + " on " + str(model_exp))
plt.plot(X_test['valueStdScaledVarForecast2'].to_numpy(),label="Predict " + str(predict_ahead) + "-step Forecast", alpha=0.6, c='red', linewidth=3)
plt.plot(X_test['valueStdScaled'].to_numpy(),label="Original Data", alpha=0.6, c='black')
plt.scatter(np.where(anomalies_ae2==True), X_test['valueStdScaledVarForecast2'].to_numpy()[np.where(anomalies_ae2==True)], 
            alpha=0.8, color='green', s=350, label="Anomalies AE")
plt.scatter(np.where(anomalies_se2==True), X_test['valueStdScaledVarForecast2'].to_numpy()[np.where(anomalies_se2==True)], 
            alpha=0.8, color='magenta', s=300, label = "Anomalies SE")
plt.scatter(np.where(anomalies_errors_ae2==True), X_test['valueStdScaledVarForecast2'].to_numpy()[np.where(anomalies_errors_ae2==True)], 
            alpha=0.8, color='blue', s=250, label = "Z-score Anomalies AE")
plt.scatter(np.where(anomalies_errors_se2==True), X_test['valueStdScaledVarForecast2'].to_numpy()[np.where(anomalies_errors_se2==True)], 
            alpha=0.8, color='cyan', s=200, label = "Z-score Anomalies SE")
plt.scatter(np.where(anomalies_errors_ae2_mod==True), X_test['valueStdScaledVarForecast2'].to_numpy()[np.where(anomalies_errors_ae2_mod==True)], 
            alpha=0.8, color='lime', s=150, label = "Modified Z-score Anomalies AE")
plt.scatter(np.where(anomalies_errors_se2_mod==True), X_test['valueStdScaledVarForecast2'].to_numpy()[np.where(anomalies_errors_se2_mod==True)], 
            alpha=0.8, color='orange', s=50, label = "Modified Z-score Anomalies SE")        
plt.legend();    
figName = f"Y-predict-anomalies-step-{predict_ahead}-with-T_{T}.png"
plt.savefig(figName, transparent=False)


In [ ]:
plt.figure(figsize=(20,15))
plt.title("Compare forecasts T=" + str(T) + " predict_ahead=" + str(predict_ahead) + " with predict 1")
plt.plot(X_test['TimeLatencyStdScaledVarForecast'],label="Counts Forecast", alpha=0.6, c='blue', linewidth=3)
plt.plot(X_test['TimeLatencyStdScaledVarForecast2'],label="Counts Forecast", alpha=0.6, c='yellow', linewidth=3)
plt.plot(X_test['TimeLatencyStdScaled'],label="Original Data", alpha=1, c='gray')
plt.legend()

In [ ]:
plt.figure(figsize=(20,15))
plt.title("Compare forecasts T=" + str(T) + " predict_ahead=" + str(predict_ahead) + " with predict 1")
plt.plot(X_test['valueStdScaledVarForecast'],label="Counts Forecast", alpha=0.6, c='green', linewidth=3)
plt.plot(X_test['valueStdScaledVarForecast2'],label="Counts Forecast n_steps", alpha=0.6, c='orange', linewidth=3)
plt.plot(X_test['valueStdScaled'],label="Original Data", alpha=0.5, c='gray')

plt.legend()
plt.savefig('Compare_forecast_T_' + str(T) + "_pred_ahead_" + str(predict_ahead) + "_with_predict_1" + ".png") 

In [ ]:
aic = calculate_aic(len(X_test['TimeLatencyStdScaled']), 
                    mean_squared_error(X_test['TimeLatencyStdScaled'], X_test['TimeLatencyStdScaledVarForecast']),
                    2
                   )

print(f"Calculating scores for TimeLatencyStdScaled with forecast T={T}, predict_ahead=1\n"
      f" R2: {r2_score(X_test['TimeLatencyStdScaled'], X_test['TimeLatencyStdScaledVarForecast'])} \n"
      f"MAE: {mean_absolute_error(X_test['TimeLatencyStdScaled'], X_test['TimeLatencyStdScaledVarForecast'])}\n"
      f"MAPE: {mean_absolute_percentage_error(X_test['TimeLatencyStdScaled'], X_test['TimeLatencyStdScaledVarForecast'])}\n"
      f"MSE: {mean_squared_error(X_test['TimeLatencyStdScaled'], X_test['TimeLatencyStdScaledVarForecast'])}\n"
      f"Pearson correlation: {np.corrcoef(X_test['TimeLatencyStdScaled'], X_test['TimeLatencyStdScaledVarForecast'])[0,1]}\n"
      f"AIC: {aic}\n"      
     )

In [ ]:
aic = calculate_aic(len(X_test['valueStdScaled']), 
                    mean_squared_error(X_test['valueStdScaled'], X_test['valueStdScaledVarForecast']),
                    2
                   )

print(f"Calculating scores for valueStdScaled with forecast T={T}, predict_ahead=1\n"
      f" R2: {r2_score(X_test['valueStdScaled'], X_test['valueStdScaledVarForecast'])} \n"
      f"MAE: {mean_absolute_error(X_test['valueStdScaled'], X_test['valueStdScaledVarForecast'])}\n"
      f"MAPE: {mean_absolute_percentage_error(X_test['valueStdScaled'], X_test['valueStdScaledVarForecast'])}\n"
      f"MSE: {mean_squared_error(X_test['valueStdScaled'], X_test['valueStdScaledVarForecast'])}\n"
      f"Pearson correlation: {np.corrcoef(X_test['valueStdScaled'], X_test['valueStdScaledVarForecast'])[0,1]}\n"
      f"AIC: {aic}\n"      
     )

In [ ]:
aic = calculate_aic(len(X_test['TimeLatencyStdScaled']), 
                    mean_squared_error(X_test['TimeLatencyStdScaled'], X_test['TimeLatencyStdScaledVarForecast2']),
                    2
                   )

print(f"Calculating scores for TimeLatencyStdScaled with forecast T={T}, predict_ahead={predict_ahead}\n"
      f" R2: {r2_score(X_test['TimeLatencyStdScaled'], X_test['TimeLatencyStdScaledVarForecast2'])} \n"
      f"MAE: {mean_absolute_error(X_test['TimeLatencyStdScaled'], X_test['TimeLatencyStdScaledVarForecast2'])}\n"
      f"MAPE: {mean_absolute_percentage_error(X_test['TimeLatencyStdScaled'], X_test['TimeLatencyStdScaledVarForecast2'])}\n"
      f"MSE: {mean_squared_error(X_test['TimeLatencyStdScaled'], X_test['TimeLatencyStdScaledVarForecast2'])}\n"
      f"Pearson correlation: {np.corrcoef(X_test['TimeLatencyStdScaled'], X_test['TimeLatencyStdScaledVarForecast2'])[0,1]}\n"
      f"AIC: {aic}\n"      
     )

In [ ]:
aic = calculate_aic(len(X_test['valueStdScaled']), 
                    mean_squared_error(X_test['valueStdScaled'], X_test['valueStdScaledVarForecast2']),
                    2
                   )
print(f"Calculating scores for valueStdScaled with forecast T={T}, predict_ahead={predict_ahead}\n"
      f" R2: {r2_score(X_test['valueStdScaled'], X_test['valueStdScaledVarForecast2'])} \n"
      f"MAE: {mean_absolute_error(X_test['valueStdScaled'], X_test['valueStdScaledVarForecast2'])}\n"
      f"MAPE: {mean_absolute_percentage_error(X_test['valueStdScaled'], X_test['valueStdScaledVarForecast2'])}\n"
      f"MSE: {mean_squared_error(X_test['valueStdScaled'], X_test['valueStdScaledVarForecast2'])}\n"
      f"Pearson correlation: {np.corrcoef(X_test['valueStdScaled'], X_test['valueStdScaledVarForecast2'])[0,1]}\n"
      f"AIC: {aic}\n"
     )

In [ ]:
results_VAR.save('results_VAR.pkl')

In [ ]:
plt.figure(figsize=(25,15))
#plt.plot(X_test['valueStdScaledForecast'],label="Counts Forecast", alpha=0.6, c='green', linewidth=3)
plt.title("Anomalies T=" + str(T) + " with predict 1")
plt.plot(X_test['valueStdScaledVarForecast'],label="Counts Forecast", alpha=0.6, c='green', linewidth=3)
plt.plot(X_test['valueStdScaled'],label="Original Data Counts", alpha=0.5, c='gray')

#plt.scatter(X_test.index[X_test['anomalies_ae_Counts']==True], 
#            X_test['valueStdScaled'][X_test['anomalies_ae_Counts']==True], alpha=0.8, color='green', s=300)
#plt.scatter(X_test.index[X_test['anomalies_se_Counts']==True], 
#            X_test['valueStdScaled'][X_test['anomalies_se_Counts']==True], alpha=0.8, color='magenta', s=100)

#plt.plot(X_test['TimeLatencyStdScaledForecast'],label="Counts Forecast", alpha=0.6, c='blue', linewidth=3)
plt.plot(X_test['TimeLatencyStdScaledVarForecast'],label="Time Resp Forecast", alpha=0.6, c='blue', linewidth=3)
plt.plot(X_test['TimeLatencyStdScaled'],label="Original Data Time Resp", alpha=1, c='gray')

#plt.scatter(X_test.index[X_test['anomalies_ae_TimeResp']==True], 
#            X_test['TimeLatencyStdScaled'][X_test['anomalies_ae_TimeResp']==True], alpha=0.8, color='green', s=300)
#plt.scatter(X_test.index[X_test['anomalies_se_TimeResp']==True], 
#            X_test['TimeLatencyStdScaled'][X_test['anomalies_se_TimeResp']==True], alpha=0.8, color='magenta', s=100)

plt.legend()
plt.savefig('Compare_T_' + str(T) + "_with_predict_1_" + ".png") 

In [ ]:
plt.figure(figsize=(25,15))
#plt.plot(X_test['valueStdScaledForecast'],label="Counts Forecast", alpha=0.6, c='green', linewidth=3)
plt.title("Anomalies T=" + str(T) + " predict_ahead=" + str(predict_ahead))
plt.plot(X_test['valueStdScaledVarForecast2'],label="Counts Forecast", alpha=0.6, c='green', linewidth=3)
plt.plot(X_test['valueStdScaled'],label="Original Data Counts", alpha=0.5, c='gray')

#plt.scatter(X_test.index[X_test['anomalies_ae_Counts2']==True], 
#            X_test['valueStdScaled'][X_test['anomalies_ae_Counts2']==True], alpha=0.8, color='green', s=300)
#plt.scatter(X_test.index[X_test['anomalies_se_Counts2']==True], 
#            X_test['valueStdScaled'][X_test['anomalies_se_Counts2']==True], alpha=0.8, color='magenta', s=100)

#plt.plot(X_test['TimeLatencyStdScaledForecast'],label="Counts Forecast", alpha=0.6, c='blue', linewidth=3)
plt.plot(X_test['TimeLatencyStdScaledVarForecast2'],label="Time Resp Forecast", alpha=0.6, c='blue', linewidth=3)
plt.plot(X_test['TimeLatencyStdScaled'],label="Original Data Time Resp", alpha=1, c='gray')

#plt.scatter(X_test.index[X_test['anomalies_ae_TimeResp2']==True], 
#            X_test['TimeLatencyStdScaled'][X_test['anomalies_ae_TimeResp2']==True], alpha=0.8, color='green', s=300)
#plt.scatter(X_test.index[X_test['anomalies_se_TimeResp2']==True], 
#            X_test['TimeLatencyStdScaled'][X_test['anomalies_se_TimeResp2']==True], alpha=0.8, color='magenta', s=100)

plt.legend()

plt.savefig('Compare_T_' + str(T) + "_pred_ahead_" + str(predict_ahead) +".png") 